# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
from sklearn.utils import shuffle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
validation_file= 'traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [33]:
import tensorflow as tf
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# y_train.shape is 2d, (50000, 1). While Keras is smart enough to handle this
# it's a good idea to flatten the array.
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)
X_train = (X_train*2.0 / 255.0) - 1.0
X_test = (X_test*2.0 / 255.0) - 1.0
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [25]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results
import numpy as np

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(np.concatenate([y_train, y_valid, y_test])))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print("Max Intensity =", X_train.max())

Number of training examples = 50000
Number of testing examples = 10000
Image data shape = (32, 32, 3)
Number of classes = 10
Max Intensity = 1.0


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [4]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import pandas as pd
import random
import numpy as np
from textwrap import wrap
import matplotlib.pyplot as plt
import os
# Visualizations will be shown in the notebook.
%matplotlib inline

# Import signnames
df = pd.read_csv('signnames.csv', index_col=0)

# Append counts of each signname in each dataset
df['n_train'] = np.bincount(y_train, minlength=n_classes)
df['n_valid'] = np.bincount(y_valid, minlength=n_classes)
df['n_test'] = np.bincount(y_test, minlength=n_classes)

# Plot numbers of each class in each set
ax = df.plot.barh(x='SignName', y=['n_train','n_valid','n_test'], figsize=(8,18),
                  title="Number of Examples by Sign Type");
plt.gcf().savefig(os.path.join('data_visualizations', 'num_examples_by_sign_type.png'), bbox_inches='tight')

# Plot 16 random training images and their labels
subplot_shape = np.array([4, 4])
indices = random.sample(range(len(X_train)), subplot_shape.prod())

def plot_image_subset(images, indices, subplot_shape, fig=None):
    """Plot images in a grid"""
    
    if fig is None:
        fig = plt.figure(figsize=(13,13))
    
    for i_ax in range(1, subplot_shape.prod()+1):
        index = indices[i_ax - 1]
        image = images[index]
        ax = fig.add_subplot(*subplot_shape, i_ax)
        plt.imshow(image.squeeze(), cmap='gray')
        wrapped = '\n'.join(wrap(df.SignName[y_train[index]], 30))
        ax.set_title(wrapped, {'fontsize': 10})
        ax.set_xticks([])
        ax.set_yticks([])
    
    return fig

fig = plot_image_subset(X_train, indices, subplot_shape)
fig.savefig(os.path.join('data_visualizations', '16_random_training_images.png'), bbox_inches='tight')



ValueError: Length of values does not match length of index

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [9]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

In [6]:
from sklearn import preprocessing
import cv2


def grayscale(image_data):
    """Applies grayscale transform using cv2 module"""
    return np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in image_data])[..., np.newaxis]


def normalize(image_data):
    """Offsets and scales the data to the interval [-1,1)"""
    return np.array([(image - image.mean()) / image.std() for image in image_data])


# Convert training data to grayscale and visualize
X_train_gray = grayscale(X_train)
#fig = plot_image_subset(X_train_gray, indices, subplot_shape)
#fig.savefig(os.path.join('data_visualizations', '16_grayscale_training_images.png'), bbox_inches='tight')

# Normalize (more specifically, standardize) training data print the mean and std of the first image
X_train_norm = normalize(grayscale(X_train))
print("X_train_norm[0].mean() = ", X_train_norm[0].mean(), " X_train_norm[0].std() = ", X_train_norm[0].std())

X_train_norm[0].mean() =  0.0  X_train_norm[0].std() =  1.0


### Model Architecture

In [49]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [34]:
import tensorflow as tf


In [35]:
from tensorflow.contrib.layers import flatten

inshape = X_train[0].shape
print(inshape)
n_classes = len(np.unique(np.concatenate([y_train, y_valid, y_test])))
print(n_classes)


def conv2d(x, W, b, strides=1, padding='VALID'):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='VALID')


def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    weights = {'wc1': tf.Variable(tf.truncated_normal((5, 5, inshape[2], 6), mu, sigma)),
               'wc2': tf.Variable(tf.truncated_normal((5, 5, 6, 16), mu, sigma)),
               'wd1': tf.Variable(tf.truncated_normal((5*5*16, 120), mu, sigma)),
               'wd2': tf.Variable(tf.truncated_normal((120, 84), mu, sigma)),
               'out': tf.Variable(tf.truncated_normal((84, n_classes), mu, sigma))}
    biases = {'bc1': tf.Variable(tf.zeros(6)),
              'bc2': tf.Variable(tf.zeros(16)),
              'bd1': tf.Variable(tf.zeros(120)),
              'bd2': tf.Variable(tf.zeros(84)),
              'out': tf.Variable(tf.zeros(n_classes))}
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    # TODO: Activation.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    #conv1 = tf.nn.dropout(conv1, keep_prob)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = maxpool2d(conv1)

    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    # TODO: Activation.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    #conv2 = tf.nn.dropout(conv2, keep_prob)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = maxpool2d(conv2)

    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = tf.matmul(fc1, weights['wd1']) + biases['bd1']
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = tf.matmul(fc1, weights['wd2']) + biases['bd2']
    
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, keep_prob)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    logits = tf.matmul(fc2, weights['out']) + biases['out']
    
    return logits, weights

(32, 32, 3)
10


### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the test set but low accuracy on the validation set implies overfitting.

In [36]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [37]:
x = tf.placeholder(tf.float32, (None,) + inshape)
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)
keep_prob = tf.placeholder(tf.float32)

In [41]:
EPOCHS = 20
BATCH_SIZE = 128
rate = 0.001
beta = 0.0001

logits, weights = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
#l2_reg_term = beta*(tf.nn.l2_loss(weights['wc1']) +
#                    tf.nn.l2_loss(weights['wc2']))
                    #tf.nn.l2_loss(weights['wd1']) +
                    #tf.nn.l2_loss(weights['wd2']) + 
                    #tf.nn.l2_loss(weights['out']))
loss_operation = tf.reduce_mean(cross_entropy)#+ l2_reg_term
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [12]:
X_train_subsets = [X_train[:BATCH_SIZE*10*m] for m in range(1, 28)]
y_train_subsets = [y_train[:BATCH_SIZE*10*m] for m in range(1, 28)]

In [42]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy, loss = sess.run([accuracy_operation, loss_operation], feed_dict={x: batch_x, y: batch_y,
                                                                                   keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
        total_loss += (loss * len(batch_x))
    return total_accuracy / num_examples, total_loss / num_examples

In [45]:
from sklearn.utils import shuffle

# Measurements use for graphing loss and accuracy
log_batch_step = 50
batches = []
loss_batch = []
train_acc_batch = []
valid_acc_batch = []
valid_acc_epoch = []

train_acc_subset = []
valid_acc_subset = []
train_loss_subset = []
valid_loss_subset = []
m = None

#for m, (X_train, y_train) in enumerate(zip(X_train_subsets, y_train_subsets)):
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)

    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for batch_i, offset in enumerate(range(0, num_examples, BATCH_SIZE)):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})

            # Log every 50 batches
            #if not batch_i % log_batch_step:
            #    # Calculate Training and Validation accuracy
            #    training_accuracy = evaluate(X_train, y_train)
            #    validation_accuracy = evaluate(X_valid, y_valid)

                # Log batches
            #    previous_batch = batches[-1] if batches else 0
            #    batches.append(log_batch_step + previous_batch)
            #    loss_batch.append(l)
            #    train_acc_batch.append(training_accuracy)
            #    valid_acc_batch.append(validation_accuracy)


        training_accuracy, training_loss = evaluate(X_train, y_train)
        validation_accuracy, validation_loss = evaluate(X_valid, y_valid)
        valid_acc_epoch.append(validation_accuracy)
        print("EPOCH {}, m={}...".format(i+1, m))
        print("Training: Accuracy = {:.6f}, Loss = {:.6f}".format(training_accuracy, training_loss))
        print("Validation: Accuracy = {:.6f}, Loss = {:.6f}".format(validation_accuracy, validation_loss))
        test_accuracy, test_loss = evaluate(X_test, y_test)
        print("Test: Accuracy = {:.6f}, Loss = {:.6f}".format(test_accuracy, test_loss))
        print()

    #saver.save(sess, './lenet')
    #print("Model saved")
        
#    train_acc_subset.append(training_accuracy)
#    valid_acc_subset.append(validation_accuracy)
#    train_loss_subset.append(training_loss)
#    valid_loss_subset.append(validation_loss)



Training...

EPOCH 1, m=None...
Training: Accuracy = 0.402747, Loss = 1.640668
Validation: Accuracy = 0.397600, Loss = 1.652063
Test: Accuracy = 0.404700, Loss = 1.644797

EPOCH 2, m=None...
Training: Accuracy = 0.460507, Loss = 1.470423
Validation: Accuracy = 0.457040, Loss = 1.487277
Test: Accuracy = 0.454200, Loss = 1.479180

EPOCH 3, m=None...
Training: Accuracy = 0.493493, Loss = 1.387094
Validation: Accuracy = 0.484640, Loss = 1.412839
Test: Accuracy = 0.487100, Loss = 1.405448

EPOCH 4, m=None...
Training: Accuracy = 0.519093, Loss = 1.313665
Validation: Accuracy = 0.508080, Loss = 1.342465
Test: Accuracy = 0.510500, Loss = 1.343280

EPOCH 5, m=None...
Training: Accuracy = 0.534773, Loss = 1.274141
Validation: Accuracy = 0.520320, Loss = 1.312793
Test: Accuracy = 0.525700, Loss = 1.310610

EPOCH 6, m=None...
Training: Accuracy = 0.554347, Loss = 1.226872
Validation: Accuracy = 0.536240, Loss = 1.267579
Test: Accuracy = 0.538000, Loss = 1.278466

EPOCH 7, m=None...
Training: Accu

AttributeError: 'NoneType' object has no attribute 'run'

In [ ]:
fig = plt.figure()
plt.plot(train_acc_subset)
plt.plot(valid_acc_subset)
plt.legend(['train','valid'])
plt.grid('on')
plt.title('Accuracy vs. Training set size')
plt.xlabel('Num_train_examples/1280')
plt.ylabel('Accuracy')
fig.savefig('figs/learn_curve_graynormdropout_epochs_{}_batchsize_{}_rate_{}.png'.format(EPOCHS, BATCH_SIZE, rate))
#for t_acc, v_acc in zip(train_acc_subset, valid_acc_subset):
#    print(t_acc, v_acc)

In [ ]:
fig = plt.figure()
plt.plot(train_loss_subset)
plt.plot(valid_loss_subset)
plt.legend(['train','valid'])
plt.grid('on')
plt.title('Loss vs. Training set size')
plt.xlabel('Num_train_examples/1280')
plt.ylabel('Loss')
fig.savefig('figs/learn_curve_graynormdropout_epochs_{}_batchsize_{}_rate_{}.png'.format(EPOCHS, BATCH_SIZE, rate))

In [ ]:
fig = plt.figure()
loss_plot = fig.add_subplot(211)
loss_plot.set_title('Loss = {}'.format(loss_batch[-1]))
loss_plot.plot(batches, loss_batch, 'g')
loss_plot.set_xlim([batches[0], batches[-1]])
acc_plot = fig.add_subplot(212)
acc_plot.set_title('Accuracy = {}'.format(valid_acc_epoch[-1]))
acc_plot.plot(batches, train_acc_batch, 'r', label='Training Accuracy')
acc_plot.plot(batches, valid_acc_batch, 'x', label='Validation Accuracy')
acc_plot.set_ylim([0, 1.0])
acc_plot.set_xlim([batches[0], batches[-1]])
acc_plot.legend(loc=4)
plt.tight_layout()
plt.show()
fig.savefig('figs/sigmoid_grayscale_epochs_{}_batchsize_{}_rate_{}.png'.format(EPOCHS, BATCH_SIZE, rate));

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the IPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 